In [ ]:
!pip install matplot lib
!pip install keras
!pip install tqdm
!pip3 install scikit-learn

In [1]:
from keras.callbacks import Callback
from tqdm import tqdm

class ProgressBar(Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']

    def on_epoch_begin(self, epoch, logs=None):
        print(f"Epoch {epoch+1}/{self.epochs}")
        self.progbar = tqdm(total=self.params['steps'])

    def on_batch_end(self, batch, logs=None):
        self.progbar.update(1)

    def on_epoch_end(self, epoch, logs=None):
        self.progbar.close()


2024-01-09 02:11:20.128268: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-09 02:11:20.163946: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-09 02:11:20.163975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-09 02:11:20.165040: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-09 02:11:20.171239: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-09 02:11:20.171683: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
import sys

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# normalize data
def normalize_data(train, test):
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	return train_norm, test_norm

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['acc'], color='blue', label='train')
	pyplot.plot(history.history['val_acc'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()


In [3]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense, Add, GlobalAveragePooling2D

def residual_block(x, filters, kernel_size=3, strides=1):
    # Shortcut connection
    shortcut = x
    
    # Main path
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding='same')(x)
    x = BatchNormalization()(x)
    
    # Skip connection
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

# Define the ResNet9 model using Keras Functional API
def define_resnet9():
    input = Input(shape=(None, None, 3))
    
    # Conv1
    x = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Conv2
    x = Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    # Res1
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=128)
    
    # Conv3
    x = Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    # Conv4
    x = Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    # Res2
    x = residual_block(x, filters=512)
    x = residual_block(x, filters=512)
    
    # Classifier
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=x)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model



In [4]:
# example of loading the cifar10 dataset
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint,LearningRateScheduler

trainX, trainY, testX, testY = load_dataset()
trainX, testX = normalize_data(trainX,testX)

trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

#model = define_model()
model = define_resnet9()

epochs = 50
max_lr = 0.01
filename = 'CNN-checkpointer.keras'

#data augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
it_train = datagen.flow(trainX, trainY, batch_size=32)
steps = int(trainX.shape[0] / 64)

#callbacks
earlystopper = EarlyStopping(monitor='val_acc', patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=5, 
                               verbose=1, mode='max', min_lr=0.00001)
checkpointer = ModelCheckpoint(filename, monitor='val_acc', verbose=1,
                               save_best_only=True)
def lr_schedule(epoch, lr):
        if epoch < epochs * 0.5:
            return max_lr
        else:
            return max_lr * 0.1
lr_scheduler = LearningRateScheduler(lr_schedule)

2024-01-09 02:11:34.129619: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-09 02:11:34.130655: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [59]:
from keras.models import load_model

model_to_load = 'models/CNN-0.829800009727478-80epochs.keras'
model = load_model(model_to_load)

In [5]:

history = model.fit(it_train, steps_per_epoch=steps, epochs=epochs, validation_data=(valX, valY), verbose=0, callbacks=[ProgressBar(),earlystopper,reduce_lr,checkpointer,lr_scheduler])
_, acc = model.evaluate(testX, testY, verbose=0)

print('> %.3f' % (acc * 100.0))
summarize_diagnostics(history)

Epoch 1/50


100%|██████████| 625/625 [06:55<00:00,  1.50it/s]


Epoch 1: val_acc improved from -inf to 0.45440, saving model to CNN-checkpointer.keras


Epoch 2/50


100%|██████████| 625/625 [06:44<00:00,  1.54it/s]

...

Epoch 3/50


100%|██████████| 625/625 [06:32<00:00,  1.59it/s]


Epoch 3: val_acc did not improve from 0.61510


Epoch 4/50


 73%|███████▎  | 456/625 [04:24<01:37,  1.74it/s]

In [34]:
epochs_to_write = 20
model.save('CNN-'+str(acc)+'-'+str(epochs_to_write)+'epochs.keras')